In [ ]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 24.5 MB/s eta 0:00:00


In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import os
import torch
import random

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## Read Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_DIR = Path("/content/drive/MyDrive/ITMO/RW'23")
DATA_DIR = BASE_DIR.joinpath("data")

FILE_NAME = "data_topic_name_distrib.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [ ]:
data = pd.read_csv(FILE)
data.head(10)

,INPUT:wordset,OUTPUT:quality,OUTPUT:topic_name,topic_name_num
0,able amaze fill stop glad daily fantastic move...,good,feelings,1
1,absolute mission church nasa mars earth truth ...,good,astrology,1
2,absolute mission church nasa mars earth truth ...,good,nasa,1
3,absolute mission church nasa mars earth truth ...,good,space,1
4,absolute mission church nasa mars earth truth ...,good,space mission,1
5,absolutely fast white rice service noodle terr...,good,customer experience,1
6,absolutely fast white super service customer r...,good,express expedition,1
7,absolutely fast white super service customer r...,good,online shopping,2
8,absolutely fast white super service customer r...,good,review,1
9,absolutely grain there's process terrible expe...,good,comment,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5766 entries, 0 to 5765
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   INPUT:wordset      5766 non-null   object
 1   OUTPUT:quality     5766 non-null   object
 2   OUTPUT:topic_name  5766 non-null   object
 3   topic_name_num     5766 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 180.3+ KB


In [ ]:
data_text = list(set(data["INPUT:wordset"]))

##Llama2-13B

In [ ]:
!nvidia-smi

Thu Jan 11 22:35:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

##Inference

In [ ]:
def generate_answer(prompt, model, device, temp, n_token):
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [ ]:
def extract_substring(input_string):
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [ ]:
def few_shot_pipeline(model, system_prompt='', instruction='', sample='', device='cpu', temp=0.1, n_token=100):
    # Construct prompt
    prompt = f"""<s>[INST]<<SYS>> {system_prompt} <</SYS>> {instruction} TEXT: {sample} TOPIC: [/INST]""".strip()
    # Generate answer

    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    # print(answer[0])
    result = extract_substring(answer[0]).strip()

    return result.replace("\n", ". ")


### Type I

In [ ]:
prompt = """<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant for labeling TEXTS. <<SYS>>
You will receive a TEXT and you should analyze it and LABEL it with one or few words. Please, make sure you to only return the LABEL and nothing more.
TEXT: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
LABEL:[/INST]""".strip()

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=100)

print(answer[0].strip())


<s>[INST] <<SYS>> You are a helpful, respectful and honest assistant for labeling TEXTS. <<SYS>>
You will receive a TEXT and you should analyze it and LABEL it with one or few words. Please, make sure you to only return the LABEL and nothing more.
TEXT: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
LABEL:[/INST]  Sure, I'd be happy to help! Based on the text you provided, I would label it as "NEWS".</s>


In [ ]:
prompt = """<s>[INST] <<SYS>> You are an assistant for topic modeling. <<SYS>>
You will receive the TEXT and you should identify one or few TOPICS of the TEXT. Provide ONLY TOPICS and nothing more.
TEXT: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
TOPIC: [/INST]""".strip()

answer = generate_answer(prompt, model, 'cuda', temp=0.1, n_token=30)

print(answer[0].strip())


<s>[INST] <<SYS>> You are an assistant for topic modeling. <<SYS>>
You will receive the TEXT and you should identify one or few TOPICS of the TEXT. Provide ONLY TOPICS and nothing more.
TEXT: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
TOPIC: [/INST]  Sure! Here are the topics for the given text:

TOPIC: Breakfast Foods</s>


##FIRST PTOMPT

In [ ]:
random.seed(42)
random.shuffle(data_text)

In [ ]:
SYSTEM_PROMPT = "You are an assistant for topic modeling."
INSTRUCTION = "You will receive the TEXT and you should identify one or few TOPICS of the TEXT. Provide ONLY TOPIC and nothing more."

SAMPLE = data_text[2]
print(SAMPLE)

res = few_shot_pipeline(model, SYSTEM_PROMPT, INSTRUCTION, SAMPLE, device="cuda", n_token=50)

res.strip()

enjoy strong morning brew blend bitter smooth packet machine starbucks perfect aftertaste prefer iced cold


"Sure! Here's the topic of the text you provided:TOPIC: Coffee</s>"

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_llama_13B_4bit_type1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)

for i in tqdm(range(len(data_text))):
# for i in tqdm(range(10)):
    SAMPLE = data_text[i]
    res_model = few_shot_pipeline(
        model,
        SYSTEM_PROMPT,
        INSTRUCTION,
        SAMPLE,
        device="cuda",
        temp=0.1,
        n_token=100
    )
    with open(RES_FILE, 'a') as file:
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [1:03:39<00:00,  1.62s/it]
